In [1]:
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import numpy as np
plt.style.use('ggplot')
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
%matplotlib inline
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split


In [2]:
data = pd.read_csv ("C:\\Users\\nilli\\Downloads\\house-prices-advanced-regression-techniques\\train.csv")

In [3]:
df= data.copy()

In [4]:
df['Total_size']= df['BsmtFinSF1']+ df['BsmtFinSF2'] + df['1stFlrSF'] +df['2ndFlrSF']+df['GarageArea']

In [5]:
def helper(x):
    if x.BldgType == "TwnhsE":
        return 'Twnhs'
    else:
        return x.BldgType

df['BldgType_clean']=df.apply(helper,axis=1)

In [6]:
df['Outdoor_Porch_Size']= df['WoodDeckSF']+df['OpenPorchSF']

In [7]:
df["AllPorchSF"] = df["OpenPorchSF"] + df["EnclosedPorch"] + df["3SsnPorch"] + df["ScreenPorch"]


In [8]:
def recent_constr(x):
    if(x.YearBuilt < x.YearRemodAdd):
        return x.YearRemodAdd
    else: 
        return x.YearBuilt

#defining intermediate column 2    
df['Recent_yr'] = df.apply(recent_constr,axis=1)

#time to sale helper defined:
def time_to_sale(x):
    return x.YrSold - x.Recent_yr

#new column defined to be used in Train
df['Age_at_sale']=df.apply(time_to_sale,axis=1)

In [9]:
#Removing the outlier
df.drop([523,1298],axis=0,inplace=True)

In [10]:
X2=df[["Neighborhood", "OverallQual", "Total_size", "KitchenQual", "Age_at_sale", "ExterQual",
                   "YrSold", "SalePrice"]]

In [11]:
X2.dtypes

Neighborhood    object
OverallQual      int64
Total_size       int64
KitchenQual     object
Age_at_sale      int64
ExterQual       object
YrSold           int64
SalePrice        int64
dtype: object

In [12]:
X2["Logprice"] = np.log(X2["SalePrice"])
X2["Logprice"]

C:\Users\nilli\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


0       12.247694
1       12.109011
2       12.317167
3       11.849398
4       12.429216
          ...    
1455    12.072541
1456    12.254863
1457    12.493130
1458    11.864462
1459    11.901583
Name: Logprice, Length: 1458, dtype: float64

In [13]:
#Dummifying Neighborhood, ExterQual, KitchenQual, and BldgType columns

from sklearn.preprocessing import OneHotEncoder

#dummify Neighborhood column
Neighborhood_ohe = OneHotEncoder( drop='first', sparse = False )
Neighborhood_enc = Neighborhood_ohe.fit_transform( X2[['Neighborhood']] )
Neighborhood_enc = pd.DataFrame( Neighborhood_enc, columns=Neighborhood_ohe.get_feature_names(['Neighborhood']) )
X2 = pd.concat( (X2.drop(["Neighborhood"], axis=1).reset_index(drop = True), Neighborhood_enc), axis = 1 )

#dummify ExterQual Column
ExterQual_ohe = OneHotEncoder( drop='first', sparse = False )
ExterQual_enc = ExterQual_ohe.fit_transform( X2[['ExterQual']] )
ExterQual_enc = pd.DataFrame( ExterQual_enc, columns=ExterQual_ohe.get_feature_names(['ExterQual']) )
X2 = pd.concat( (X2.drop(["ExterQual"], axis=1).reset_index(drop = True), ExterQual_enc), axis = 1 )


# dummify KitchenQual column
from sklearn.preprocessing import OneHotEncoder
KitchenQual_ohe = OneHotEncoder( drop='first', sparse = False )
KitchenQual_enc = KitchenQual_ohe.fit_transform( X2[['KitchenQual']] )
KitchenQual_enc = pd.DataFrame( KitchenQual_enc, columns=KitchenQual_ohe.get_feature_names(['KitchenQual']) )
X2 = pd.concat( (X2.drop(["KitchenQual"], axis=1).reset_index(drop = True), KitchenQual_enc), axis = 1 )

# #dummify Building Type
# BldgType_clean_ohe = OneHotEncoder( drop='first', sparse = False )
# BldgType_clean_enc = BldgType_clean_ohe.fit_transform( X2[['BldgType_clean']] )
# BldgType_clean_enc = pd.DataFrame( BldgType_clean_enc, columns=BldgType_clean_ohe.get_feature_names(['BldgType_clean']) )
# X2 = pd.concat( (X2.drop(["BldgType_clean"], axis=1).reset_index(drop = True), BldgType_clean_enc), axis = 1 )
# X2.head()



X2.head()

,OverallQual,Total_size,Age_at_sale,YrSold,SalePrice,Logprice,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,...,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,ExterQual_Fa,ExterQual_Gd,ExterQual_TA,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_TA
0,7,2964,5,2008,208500,12.247694,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,6,2700,31,2007,181500,12.109011,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
2,7,2880,6,2008,223500,12.317167,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,7,2575,36,2006,140000,11.849398,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
4,8,3689,8,2008,250000,12.429216,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


In [14]:
y = X2[["Logprice"]]
X = X2.drop(["SalePrice", "Logprice"], axis=1)
print(X.shape)
print(y.shape)

(1458, 34)
(1458, 1)


In [15]:
np.random.seed(0)
from sklearn.model_selection import train_test_split

Xtrain, Xtest, ytrain, ytest = train_test_split( X,y, test_size = 0.2  )
print(Xtrain.shape)
print(ytrain.shape)

(1166, 34)
(1166, 1)


In [16]:
Xtrain.head()

,OverallQual,Total_size,Age_at_sale,YrSold,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,...,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,ExterQual_Fa,ExterQual_Gd,ExterQual_TA,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_TA
1170,6,2356,50,2008,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
75,4,1747,36,2009,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
390,5,2383,58,2008,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1282,6,1824,39,2010,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
682,9,3429,5,2007,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


In [17]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
Xtrain = pd.DataFrame( ss.fit_transform(Xtrain), columns = Xtrain.columns)
Xtrain.head()

,OverallQual,Total_size,Age_at_sale,YrSold,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,...,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,ExterQual_Fa,ExterQual_Gd,ExterQual_TA,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_TA
0,-0.062929,-0.116190,1.288237,0.126946,-0.029298,-0.101974,-0.204943,-0.144968,-0.341845,-0.193304,...,-0.254637,-0.117954,-0.175915,-0.083117,-0.09759,-0.710293,0.774242,-0.170664,-0.814456,0.981307
1,-1.545262,-0.839015,0.612514,0.874518,-0.029298,-0.101974,-0.204943,-0.144968,-0.341845,-0.193304,...,-0.254637,-0.117954,-0.175915,-0.083117,-0.09759,-0.710293,0.774242,-0.170664,-0.814456,0.981307
2,-0.804096,-0.084144,1.674364,0.126946,-0.029298,-0.101974,-0.204943,-0.144968,-0.341845,-0.193304,...,-0.254637,-0.117954,-0.175915,-0.083117,-0.09759,-0.710293,0.774242,-0.170664,-0.814456,0.981307
3,-0.062929,-0.747623,0.757311,1.622090,-0.029298,-0.101974,-0.204943,-0.144968,-0.341845,-0.193304,...,-0.254637,-0.117954,-0.175915,-0.083117,-0.09759,-0.710293,0.774242,-0.170664,-0.814456,0.981307
4,2.160570,1.157357,-0.883730,-0.620626,-0.029298,-0.101974,-0.204943,-0.144968,2.925299,-0.193304,...,-0.254637,-0.117954,-0.175915,-0.083117,-0.09759,1.407870,-1.291585,-0.170664,1.227814,-1.019049


In [18]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV

In [22]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
lasso2 = Lasso(warm_start = True, max_iter = 1e7)

params = {'alpha':np.logspace(-6, 0, 100)}
grid_search_lasso = GridSearchCV( estimator=lasso2, param_grid=params, cv=5 )
grid_search_lasso.fit(Xtrain,ytrain)

GridSearchCV(cv=5, error_score=nan,
             estimator=Lasso(alpha=1.0, copy_X=True, fit_intercept=True,
                             max_iter=10000000.0, normalize=False,
                             positive=False, precompute=False,
                             random_state=None, selection='cyclic', tol=0.0001,
                             warm_start=True),
             iid='deprecated', n_jobs=None,
             param_grid={'alpha': array([1.00000000e-06, 1.14975700e-06, 1.32194115e-06, 1.51991108e-06,
       1.74752840...
       7.05480231e-02, 8.11130831e-02, 9.32603347e-02, 1.07226722e-01,
       1.23284674e-01, 1.41747416e-01, 1.62975083e-01, 1.87381742e-01,
       2.15443469e-01, 2.47707636e-01, 2.84803587e-01, 3.27454916e-01,
       3.76493581e-01, 4.32876128e-01, 4.97702356e-01, 5.72236766e-01,
       6.57933225e-01, 7.56463328e-01, 8.69749003e-01, 1.00000000e+00])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, ver

In [23]:
grid_search_lasso.best_params_

{'alpha': 0.0003511191734215131}

In [24]:
grid_search_lasso.cv_results_

{'mean_fit_time': array([0.01338143, 0.00642662, 0.00811687, 0.00858622, 0.00734839,
        0.0079844 , 0.0067131 , 0.00801077, 0.00731006, 0.00701318,
        0.00679584, 0.00725007, 0.00618162, 0.00631647, 0.00666537,
        0.00684462, 0.00574393, 0.0073133 , 0.00628281, 0.00572653,
        0.0066236 , 0.00597997, 0.006214  , 0.00507288, 0.00325904,
        0.00487442, 0.00519471, 0.00274019, 0.00582395, 0.00604296,
        0.00520668, 0.00363584, 0.00408416, 0.00405684, 0.00457363,
        0.00194497, 0.00468035, 0.00212703, 0.00313292, 0.00206037,
        0.00386662, 0.00375004, 0.00283737, 0.00247855, 0.00220189,
        0.00302138, 0.00276971, 0.0011867 , 0.00192351, 0.00121708,
        0.00354266, 0.00186152, 0.00386686, 0.00248199, 0.00280285,
        0.00334363, 0.00253701, 0.0020812 , 0.00248623, 0.00214901,
        0.00163155, 0.00183644, 0.00176754, 0.00165763, 0.00270033,
        0.00198197, 0.00180173, 0.00202408, 0.001615  , 0.00214944,
        0.00140486, 0.00201025,

In [25]:
grid_search_lasso.best_score_

0.8635565398871196

In [26]:
lasso2.fit(Xtest, ytest).coef_
lst = []
for i in (list(zip(lasso2.fit(Xtest, ytest).coef_, Xtest.columns))):
    lst.append(i)
lst

[(0.0, 'OverallQual'),
 (0.0003611951405115372, 'Total_size'),
 (-0.004290704226077018, 'Age_at_sale'),
 (0.0, 'YrSold'),
 (-0.0, 'Neighborhood_Blueste'),
 (-0.0, 'Neighborhood_BrDale'),
 (-0.0, 'Neighborhood_BrkSide'),
 (-0.0, 'Neighborhood_ClearCr'),
 (0.0, 'Neighborhood_CollgCr'),
 (0.0, 'Neighborhood_Crawfor'),
 (-0.0, 'Neighborhood_Edwards'),
 (0.0, 'Neighborhood_Gilbert'),
 (-0.0, 'Neighborhood_IDOTRR'),
 (-0.0, 'Neighborhood_MeadowV'),
 (-0.0, 'Neighborhood_Mitchel'),
 (-0.0, 'Neighborhood_NAmes'),
 (0.0, 'Neighborhood_NPkVill'),
 (-0.0, 'Neighborhood_NWAmes'),
 (0.0, 'Neighborhood_NoRidge'),
 (0.0, 'Neighborhood_NridgHt'),
 (-0.0, 'Neighborhood_OldTown'),
 (-0.0, 'Neighborhood_SWISU'),
 (-0.0, 'Neighborhood_Sawyer'),
 (-0.0, 'Neighborhood_SawyerW'),
 (0.0, 'Neighborhood_Somerst'),
 (0.0, 'Neighborhood_StoneBr'),
 (-0.0, 'Neighborhood_Timber'),
 (0.0, 'Neighborhood_Veenker'),
 (-0.0, 'ExterQual_Fa'),
 (0.0, 'ExterQual_Gd'),
 (-0.0, 'ExterQual_TA'),
 (-0.0, 'KitchenQual_Fa'),
 (0

In [28]:
from sklearn.metrics import mean_squared_error
print("R^2: {}".format(lasso2.score(Xtest, ytest)))
mse=(mean_squared_error(ytest, lasso2.predict(Xtest)))
print("Mean Squared Error: {}".format(mse))
rmse = np.sqrt(mean_squared_error(ytest, lasso2.predict(Xtest)))
print("Root Mean Squared Error: {}".format(rmse))  

R^2: 0.7712944225850736
Mean Squared Error: 0.03875977739663793
Root Mean Squared Error: 0.19687502989622105


In [31]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
lasso3 = Lasso(warm_start = True, max_iter = 1e7)

params = {'alpha':np.linspace( 0.000351119173421, 0.00002, 100)}
grid_search_lasso = GridSearchCV( estimator=lasso3, param_grid=params, cv=5 )
grid_search_lasso.fit(Xtrain,ytrain)

GridSearchCV(cv=5, error_score=nan,
             estimator=Lasso(alpha=1.0, copy_X=True, fit_intercept=True,
                             max_iter=10000000.0, normalize=False,
                             positive=False, precompute=False,
                             random_state=None, selection='cyclic', tol=0.0001,
                             warm_start=True),
             iid='deprecated', n_jobs=None,
             param_grid={'alpha': array([3.51119173e-04, 3.47774535e-04, 3.44429897e-04, 3.41085259e-04,
       3.37740621...
       8.35481242e-05, 8.02034861e-05, 7.68588480e-05, 7.35142098e-05,
       7.01695717e-05, 6.68249336e-05, 6.34802955e-05, 6.01356574e-05,
       5.67910193e-05, 5.34463812e-05, 5.01017430e-05, 4.67571049e-05,
       4.34124668e-05, 4.00678287e-05, 3.67231906e-05, 3.33785525e-05,
       3.00339143e-05, 2.66892762e-05, 2.33446381e-05, 2.00000000e-05])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, ver

In [32]:
grid_search_lasso.best_params_

{'alpha': 0.000351119173421}

In [33]:
grid_search_lasso.best_score_

0.8635565398871196

In [34]:
lasso3.fit(Xtest, ytest).coef_
lst = []
for i in (list(zip(lasso3.fit(Xtest, ytest).coef_, Xtest.columns))):
    lst.append(i)
lst

[(0.0, 'OverallQual'),
 (0.0003611951405115372, 'Total_size'),
 (-0.004290704226077018, 'Age_at_sale'),
 (0.0, 'YrSold'),
 (-0.0, 'Neighborhood_Blueste'),
 (-0.0, 'Neighborhood_BrDale'),
 (-0.0, 'Neighborhood_BrkSide'),
 (-0.0, 'Neighborhood_ClearCr'),
 (0.0, 'Neighborhood_CollgCr'),
 (0.0, 'Neighborhood_Crawfor'),
 (-0.0, 'Neighborhood_Edwards'),
 (0.0, 'Neighborhood_Gilbert'),
 (-0.0, 'Neighborhood_IDOTRR'),
 (-0.0, 'Neighborhood_MeadowV'),
 (-0.0, 'Neighborhood_Mitchel'),
 (-0.0, 'Neighborhood_NAmes'),
 (0.0, 'Neighborhood_NPkVill'),
 (-0.0, 'Neighborhood_NWAmes'),
 (0.0, 'Neighborhood_NoRidge'),
 (0.0, 'Neighborhood_NridgHt'),
 (-0.0, 'Neighborhood_OldTown'),
 (-0.0, 'Neighborhood_SWISU'),
 (-0.0, 'Neighborhood_Sawyer'),
 (-0.0, 'Neighborhood_SawyerW'),
 (0.0, 'Neighborhood_Somerst'),
 (0.0, 'Neighborhood_StoneBr'),
 (-0.0, 'Neighborhood_Timber'),
 (0.0, 'Neighborhood_Veenker'),
 (-0.0, 'ExterQual_Fa'),
 (0.0, 'ExterQual_Gd'),
 (-0.0, 'ExterQual_TA'),
 (-0.0, 'KitchenQual_Fa'),
 (0

In [35]:
from sklearn.metrics import mean_squared_error
print("R^2: {}".format(lasso3.score(Xtest, ytest)))
mse=(mean_squared_error(ytest, lasso3.predict(Xtest)))
print("Mean Squared Error: {}".format(mse))
rmse = np.sqrt(mean_squared_error(ytest, lasso3.predict(Xtest)))
print("Root Mean Squared Error: {}".format(rmse))  

R^2: 0.7712944225850736
Mean Squared Error: 0.03875977739663793
Root Mean Squared Error: 0.19687502989622105
